In [1]:
import yfinance as yf
import talib
import numpy as np
from package import indicator as ind
import pandas as pd
from pandas_datareader import data as pdr
import random
from sklearn.decomposition import PCA
import sklearn.metrics
from genetic_selection import GeneticSelectionCV
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import datetime as datetime

import warnings
from package import euklid_regressor as eur
warnings.filterwarnings("ignore")

random.seed(42)
stock = "NSQ"

In [2]:
#import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)
# download dataframe
start = datetime.datetime(2000,8,23)
end = datetime.datetime(2022,4,1)
data = pdr.get_data_yahoo("NQ=F", start=start, end=end)


[*********************100%***********************]  1 of 1 completed


In [3]:
data['vwap'] = eur.vwap(data['Close'], data['Volume'])
data['SO'] = eur.Stochastic_Oscillator(data['Close'])
data["ROC_1"] =  talib.ROC(data['Close'], timeperiod=1)
data["ROC_2"] =  talib.ROC(data['Close'], timeperiod=2)
data["ROC_3"] =  talib.ROC(data['Close'], timeperiod=3)
data["ROC_4"] =  talib.ROC(data['Close'], timeperiod=4)
data["ROC_5"] =  talib.ROC(data['Close'], timeperiod=5)
data["ROC_6"] =  talib.ROC(data['Close'], timeperiod=6)
data["ROC_7"] =  talib.ROC(data['Close'], timeperiod=7)
data["ROC_30"] =  talib.ROC(data['Close'], timeperiod=30)
data["ROC_60"] =  talib.ROC(data['Close'], timeperiod=60)
data['Momentum'] = ind.momentum(data)
data['Return'] = data.Close.pct_change()
data['1 Day ROI']=ind.ROI(data,1)
data['2 Day ROI']=ind.ROI(data,2)
data['3 Day ROI']=ind.ROI(data,3)
data['4 Day ROI']=ind.ROI(data,4)
data['5 Day ROI']=ind.ROI(data,5)
data['6 Day ROI']=ind.ROI(data,6)
data['7 Day ROI']=ind.ROI(data,7)
data['10 Day ROI']=ind.ROI(data,10)
data['20 Day ROI']=ind.ROI(data,20)
data['30 Day ROI']=ind.ROI(data,30)
data['RSI_1'] = eur.rsi(data['Close'],1)
data['RSI_2'] = eur.rsi(data['Close'],2)
data['RSI_3'] = eur.rsi(data['Close'],3)
data['RSI_4'] = eur.rsi(data['Close'],4)
data['RSI_5'] = eur.rsi(data['Close'],5)
data['RSI_6'] = eur.rsi(data['Close'],6)
data['RSI_7'] = eur.rsi(data['Close'],7)
data['RSI_14'] = eur.rsi(data['Close'],14)
data['RSI_30'] = eur.rsi(data['Close'],30)
data['RSI_60'] = eur.rsi(data['Close'],60)
data['EMA_12'] = ind.EMA(data, 12)
data['EMA_26'] = ind.EMA(data, 26)
data['Up down'] = ind.clas(data, .01)
eur.macd(data,data['Close'])
#Add SRSI to our dataframe
data['SRSI_10'] = ind.SRSI(data,10)
data['SRSI_14'] = ind.SRSI(data,14)
data['SRSI_30'] = ind.SRSI(data,30)
#data['Williams_1'] = ind.Williams(data,1)
#data['Williams_2'] = ind.Williams(data,2)
#data['Williams_3'] = ind.Williams(data,3)
data['Williams_14'] = ind.Williams(data,14)
data['ATR_14'] = ind.ATR(data,14) 
data['CCI'] = ind.CCI(data,20) 
#data['SAR'] = talib.SAR(data['High'], data['Low'], acceleration=0.02, maximum=0)
#data['AROONOSC_7'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=7)
#data['AROONOSC_14'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=14)
#data['AROONOSC_21'+stock] = talib.AROONOSC(data['High'], data['Low'], timeperiod=21)
data["APO"] = talib.APO(data['Close'], fastperiod=12, slowperiod=26, matype=0)
differenced = (data['Close'] - data['Close'].shift(1))/(data['Close'].shift(1))[1:]
previous_differenced = differenced.shift(1)
data["Differenced"] = differenced
data["Previous_differenced"] = previous_differenced

data=data.drop(['Adj Close','Volume'], axis=1)
data = data.dropna()


# Cleaning

In [4]:
data = data.dropna()
data

,Open,High,Low,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,...,MACDs_12_26_9,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced
Date,,,,,,,,,,,,,,,,,,,,,
2000-12-12,2984.5,2984.50,2854.50,2867.500000,2665.908970,74.257426,-3.920255,4.634191,7.437242,4.748858,...,-134.633355,1,0.837098,0,-22.767857,190.971945,47.089948,-146.153846,-0.039203,0.089035
2000-12-13,2899.0,2964.00,2738.50,2744.000000,2654.580134,47.084708,-4.306888,-8.058301,0.127714,2.810041,...,-124.584821,0.979347,0.838715,0,-44.821429,195.575686,14.299695,-127.951923,-0.043069,-0.039203
2000-12-14,2747.5,2787.50,2614.00,2624.000000,2652.298097,20.682068,-4.373178,-8.491718,-12.079075,-4.251049,...,-117.707148,0.842195,0.558604,-0.219342,-66.25,192.632261,-43.806646,-102.538462,-0.043732,-0.043069
2000-12-15,2590.0,2640.00,2556.00,2583.280029,2641.971177,11.722779,-1.551828,-5.857142,-9.911769,-13.443457,...,-113.558835,0.762985,0.551167,-0.228483,-73.521423,178.147959,-79.510177,-79.724614,-0.015518,-0.043732
2000-12-18,2619.5,2692.00,2553.00,2608.500000,2626.291895,17.271727,0.976277,-0.590701,-4.938047,-9.032258,...,-110.677498,0.800916,0.741519,-0.028719,-69.017857,172.928232,-60.595725,-55.894486,0.009763,-0.015518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-25,14765.0,14840.50,14572.50,14755.750000,13905.922231,99.534410,-0.054187,2.137122,0.694350,2.680839,...,-92.624449,0.802474,0.691443,0.411403,-4.468758,378.996698,113.314284,87.327994,-0.000542,0.021925
2022-03-28,14756.0,14997.00,14658.75,14985.250000,14039.538800,100.000000,1.555326,1.500296,3.725687,2.260475,...,-47.738639,0.903672,0.722857,0.480099,-0.572333,373.563805,119.113994,172.374468,0.015553,-0.000542
2022-03-29,14981.0,15268.75,14956.50,15237.750000,14223.019253,100.000000,1.684990,3.266523,3.210566,5.473455,...,0.030361,0.897086,0.70502,0.50077,-1.333477,365.388631,132.292988,286.760686,0.016850,0.015553


# Trying to add also PCA (Feature augmentation)

In [5]:
y = data["Up down"]
normalized = ind.normalize(data.drop(columns=["Up down", "Differenced"]))
X = normalized


In [6]:
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(X)
data["principalDf"] = principalComponents

# Save data

In [12]:
data=data.drop(data.index[0:30])
#Remove the last row of data because class has value 'N'
data=data.drop(data.index[-1])

data.to_csv("data/"+stock+".csv")

# Feature selection with GANA

In [8]:
#Remove the first 30 index which could have a value 'N'
data=data.drop(data.index[0:30])
#Remove the last row of data because class has value 'N'
data=data.drop(data.index[-1])

data=data.drop(['High','Low'], axis=1)
data

,Open,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,ROC_5,ROC_6,...,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced,principalDf
Date,,,,,,,,,,,,,,,,,,,,,
2001-03-12,1817.50,1691.000000,1995.879864,0.000000,-6.626173,-13.103803,-14.961026,-14.832536,-11.558577,-9.885425,...,0.156335,0.105861,-0.048387,-96.993988,122.14487,-131.639933,-198.070513,-0.066262,-0.069373,19.474956
2001-03-13,1714.00,1783.000000,1985.200624,22.493888,5.440568,-1.546107,-8.376156,-10.334423,-10.198942,-6.746862,...,0.330422,0.26808,0.042038,-75.514874,121.325554,-110.344964,-193.727564,0.054406,-0.066262,10.547519
2001-03-14,1792.50,1752.500000,1975.253627,15.036675,-1.710600,3.636901,-3.230260,-9.943474,-11.868242,-11.735079,...,0.347152,0.260682,0.032218,-82.494279,119.482147,-98.937964,-194.666667,-0.017106,0.054406,11.238963
2001-03-15,1761.50,1689.000000,1964.572118,0.000000,-3.623395,-5.272013,-0.118273,-6.736610,-13.206578,-15.061604,...,0.253714,0.216865,0.042259,-97.025172,120.884527,-99.130111,-187.192308,-0.036234,-0.017106,14.564286
2001-03-16,1693.50,1663.949951,1922.594039,0.000000,-1.483129,-5.052785,-6.676952,-1.599648,-8.119826,-14.493836,...,0.28743,0.147928,0.010025,-101.278813,112.033257,-112.631793,-177.963784,-0.014831,-0.036234,14.194651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-23,14654.00,14447.000000,13793.821985,87.130867,-1.412584,0.532341,2.495088,2.375680,3.540457,7.398666,...,0.640338,0.523453,0.379605,-13.954487,405.276669,113.361254,-110.561430,-0.014126,0.019728,-4.593364
2022-03-24,14462.75,14763.750000,13839.410958,100.000000,2.192497,0.748942,2.736509,4.742290,4.620263,5.810578,...,0.717648,0.591346,0.380847,-0.328632,396.073113,117.456985,8.235045,0.021925,-0.014126,-6.861921
2022-03-25,14765.00,14755.750000,13905.922231,99.534410,-0.054187,2.137122,0.694350,2.680839,4.685533,4.563573,...,0.802474,0.691443,0.411403,-4.468758,378.996698,113.314284,87.327994,-0.000542,0.021925,-6.255217


In [9]:
y = data["Up down"]
y = y.astype('int')
#Normalize the data that we have filtered
normalized = ind.normalize(data.drop(columns=["Up down","Differenced"]))
X = normalized


In [10]:
random.seed(42)
np.random.seed(42)

estimator = DecisionTreeClassifier()
model = GeneticSelectionCV(
        estimator,
        cv=3, 
        verbose=1,
        scoring="accuracy", 
        max_features=30,
        n_population=500,
        crossover_proba=0.5,
        mutation_proba=0.2, 
        n_generations=200,
        crossover_independent_proba=0.5,
        mutation_independent_proba=0.15,
        tournament_size=4,
        n_gen_no_change=10,
        caching=True, 
        n_jobs=-1)

model = model.fit(X, y)
print('Features:', X.columns[model.support_])

Selecting features with genetic algorithm.
gen	nevals	avg                               	std                            	min                            	max                               
0  	500   	[  0.508892  15.436      0.020437]	[ 0.00794   8.682966  0.009627]	[ 0.476042  1.        0.000479]	[  0.528663  30.         0.041739]
1  	299   	[-39.4887    17.464     40.021035]	[ 631.221756    6.945841  631.188021]	[-10000.            1.            0.001876]	[     0.528663     32.        10000.      ]
2  	262   	[-19.485095  18.304     20.020263]	[ 446.789208    6.289959  446.765251]	[-10000.            1.            0.001829]	[     0.528663     31.        10000.      ]
3  	293   	[-39.48506   18.808     40.019953]	[ 631.221987    6.564079  631.188089]	[-10000.            1.            0.001567]	[     0.533925     31.        10000.      ]
4  	282   	[-59.484461  19.524     60.019391]	[ 772.309437    6.412911  772.267877]	[-10000.            1.            0.004965]	[     0.533925     32. 

In [11]:
random.seed(42)
np.random.seed(42)

estimator = linear_model.LogisticRegression(C=1e-05, penalty='none')
model = GeneticSelectionCV(
        estimator,
        cv=3, 
        verbose=1,
        scoring="accuracy", 
        max_features=30,
        n_population=1000,
        crossover_proba=0.10,
        mutation_proba=0.2, 
        n_generations=200,
        crossover_independent_proba=0.05,
        mutation_independent_proba=0.05,
        tournament_size=5,
        n_gen_no_change=10,
        caching=True, 
        n_jobs=-1)

model = model.fit(X, y)
print('Features:', X.columns[model.support_])

Selecting features with genetic algorithm.
gen	nevals	avg                               	std                            	min                            	max                               
0  	1000  	[  0.534575  15.348      0.010727]	[ 0.005758  8.652104  0.006152]	[ 0.518885  1.        0.000121]	[  0.5467    30.         0.031219]
1  	248   	[ 0.540879  7.786     0.004796]   	[ 0.003488  6.024135  0.004103]	[ 0.522832  1.        0.000121]	[  0.547265  30.         0.02078 ]
2  	276   	[ 0.54352   4.817     0.002278]   	[ 0.002098  2.878804  0.002448]	[ 0.530537  1.        0.000121]	[  0.547265  16.         0.015315]
3  	289   	[-9.456025  5.462    10.00249 ]   	[ 316.086823    2.447153  316.069534]	[-10000.            0.            0.000121]	[     0.548017     14.        10000.      ]
4  	324   	[ 0.545405  5.697     0.00304 ]   	[ 0.002333  2.029579  0.002005]      	[ 0.523959  1.        0.000121]            	[  0.548956  13.         0.01807 ]         
5  	281   	[ 0.546153  5.612     